In [ ]:
import gensim
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, punkt
from gensim.models import KeyedVectors

#setting up libraries
import pandas as pd
pd.set_option('display.max_columns', 50)
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time

In [ ]:
df = pd.read_pickle("/home/kno5cac/git/publicrd/data/prd/Paper/FR_meta_and_final_tokens_23DEC21.pkl")


In [ ]:
df = df[~df.DEPARTMENT.isnull()]

In [ ]:
df_nsf = df.loc[df['DEPARTMENT'] == 'HHS']
df_nsf = df_nsf.iloc[0:10000,]

In [ ]:
df_nasa = df.loc[df['DEPARTMENT'] == 'NSF']
df_nasa = df_nasa.iloc[0:10000,]

In [ ]:
frames = [df_nsf, df_nasa]
df_split = pd.concat(frames)
#df_split.head()

In [ ]:
df_split= df_split.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [ ]:
data = df_split['ABSTRACT']

In [ ]:
#print(data)

In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
tagged_df = pd.DataFrame(tagged_data)

In [ ]:
#print(tagged_df)

In [ ]:
df_split['words'] = tagged_df['words']
df_split['tags'] = tagged_df['tags']
#df_split.head()

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=5, epochs=100) #min_count=2 - min number of times it shows up
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
model.save("d2v.model")

In [ ]:
model = Doc2Vec.load("d2v.model")

In [ ]:
vector = []
for i in range(0, len(df_split)):
    vectorthis = (model.dv[df_split['tags'][i]]).tolist()
    vector.append(vectorthis)

In [ ]:
mat = []
for i in vector:
    #dont use this loop
    for x in i:
        mat.append(x)

In [ ]:
training = pd.DataFrame(mat)

In [ ]:
trainingnp = training.to_numpy()

In [ ]:
training_data = df_split.sample(frac=0.85, random_state=25)
testing_data = df_split.drop(training_data.index)
#print(training_data)

In [ ]:
train_id = training_data.index
test_id = testing_data.index

In [ ]:
#print(test_id)

In [ ]:
x_train = trainingnp[train_id]
x_test = trainingnp[test_id]

In [ ]:
df_split['index'] = df_split.index
y_test = df_split.loc[df_split["index"].isin(test_id)]
y_test = y_test['DEPARTMENT']

In [ ]:
df_split['index'] = df_split.index
y_train = df_split.loc[df_split["index"].isin(train_id)]
y_train = y_train['DEPARTMENT']

In [ ]:
import math

In [ ]:
neighbors = int(math.sqrt(len(df_split.index)))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 25)
classifier.fit(x_train, y_train)

In [ ]:
y_pred = classifier.predict(x_test)

In [ ]:
import scipy.sparse

In [ ]:
predictions = df_split.loc[test_id,]
predictions['PREDICTIONS'] = y_pred

In [ ]:
predictions = predictions[["index","DEPARTMENT", "PREDICTIONS","ABSTRACT"]]

In [ ]:
predictions 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
error = []

# Calculating error for K values between 1 and 40
for i in range(1, 40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)
    pred_i = knn.predict(x_test)
    error.append(np.mean(pred_i != y_test))

In [ ]:
#https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/

In [ ]:
#just to be sure we got all the sentences
print(len(train_list_of_sent))
print(len(test_list_of_sent))
print(len(crossvalidation_list_of_sent))
w2v_vector=w2v_model.wv.vectors
w2v_vector.shape
def find_word2vec(list_of_sent):
    w2v=[]
    for sent in tqdm(list_of_sent):#for each sentence
        sent_vector=np.zeros(300)#create 300 dimensions of zeros
        for word in sent:#for each word in sentence
            if word in w2v_model.wv.vocab:#if word exists in word2vec model
                vec=w2v_model.wv[word]#get the vector representation of the word
                sent_vector+=vec#add the vector to sent_vector
        #after adding all the word vectors in the sentence add the vector that now represents 
        #the whole sentence to vectors_of_sentences
        w2v.append(sent_vector)
    return w2v
#word2vec representation of train_data
train_w2v=find_word2vec(train_list_of_sent)
crossvalidation_w2v=find_word2vec(crossvalidation_list_of_sent)
test_w2v=find_word2vec(test_list_of_sent)